In [1]:
import pandas as pd
from os.path import join
from glob import glob
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm

dataroot  = '/media/juma/data/net_intrusion/IDS18/CSVs/WS_l'

In [2]:
df = None
columns = ['Flow ID','Timestamp', 'TimestampMCS','Flow Duration','FIN Flag Cnt' ]
for fn in glob(join(dataroot,'*Meter.csv')):
    df = pd.read_csv(fn, usecols=columns)

In [3]:
df['Flow Duration Sec']=df['Flow Duration']/10**6
df['TimestampSec']=df['TimestampMCS']/10**6

df.shape

(7594364, 7)

In [4]:
gdata = df.groupby(by=['Flow ID'], as_index=False)

In [5]:
def convert_to_datetime(arr):
    fmt = '%H:%M'
    offset = timedelta(hours=11,minutes=36)
    arr_str = []
    for t in arr:
        dt = datetime.fromtimestamp(t/1000000.)
        dt = dt-offset
        arr_str.append(dt.strftime(fmt))
    return arr_str

In [6]:
res = gdata['TimestampMCS'].agg({'count':'count'})
res = res.sort_values(by='count',ascending=False)

In [27]:
fin_count = gdata['FIN Flag Cnt'].agg({'#flags':'sum'})
fin_count = fin_count.sort_values(by='#flags',ascending=False)

In [35]:
fin_count.head(6)

,Flow ID,#flags
3020852,172.31.67.9-31.13.65.7-49608-443-6,12
1703304,172.31.66.114-8.43.72.21-49935-443-6,12
1165714,172.31.65.29-8.43.72.21-51157-443-6,12
3032248,172.31.67.92-151.101.34.109-50089-443-6,11
2513047,172.31.67.12-151.101.34.109-50719-443-6,11
2431563,172.31.67.107-151.101.32.175-51177-443-6,11


In [8]:
begin_time = df['TimestampMCS'].min()
end_time = df['TimestampMCS'].max()

time_labels = np.linspace(begin_time,end_time,6)
lindices = convert_to_datetime(time_labels)

In [9]:
print(time_labels,lindices)

[1.51869740e+15 1.51882543e+15 1.51895347e+15 1.51908151e+15
 1.51920955e+15 1.51933758e+15] ['09:47', '21:21', '08:55', '20:29', '08:03', '19:37']


In [30]:
record_counts = []
flow_duration = []
flowids = []
fin_counts = []
for row in tqdm(fin_count.head(6).itertuples()):
    #print(row[1],row[2],end='\t')
    frame = gdata.get_group(row[1]).sort_values(by=['TimestampMCS'])    
    if row[2]==1:
        break
    fin_occurance= frame['FIN Flag Cnt'].sum()

    hdur = (frame['TimestampMCS'].max() - frame['TimestampMCS'].min())/10**6/60//60    
    
    fin_counts.append(fin_occurance)
    flowids.append(row[1])
    record_counts.append(row[2])
    flow_duration.append(hdur)
pd.DataFrame({'Flow ID':flowids,'# records': record_counts, 'Duration in Hour': flow_duration,'FIN flags':fin_counts}).to_csv(join(dataroot,'record_count_with_fin.csv'))


6it [00:00, 663.95it/s]


In [18]:
filename = '/media/juma/data/net_intrusion/IDS18/CSVs/WS_l/record_count.csv'
df2 = pd.read_csv(filename)

In [19]:
df2['# records'].sum()

4823735

In [20]:
df2.shape

(1813352, 4)

In [21]:
dur/10**6

NameError: name 'dur' is not defined

In [ ]:
dur/10**6/60

In [ ]:
dur/10**6/60/60

In [ ]:
markers = ['>','*','<','o','^','d','D','>','*','<','o','^','d','D']
num_markers = len(markers)
figure(figsize=(25,12))
font = {'size'   : 25}
plt.rc('font', **font)
plt.rcParams["axes.linewidth"]  = 2.5
plt.grid()
counter=0
for row in res.head(1).itertuples():
    print(row[1],row[2],end='\t')
    frame = gdata.get_group(row[1]).sort_values(by=['TimestampMCS'])
    frame.head()
    plt.scatter((frame['TimestampSec']%(60*60*12)),y=frame['Flow Duration Sec'],s=500,marker=markers[counter%num_markers],
                facecolors='none',edgecolors=np.random.rand(3,),linewidth=3)
    counter+=1
plt.ylabel('Flow Duration (sec)')
plt.xlim(1760,1900)
#plt.xticks(time_labels,lindices)

In [ ]:
plt.show()

1813291it [32:42, 1031.98it/s]